In [ ]:
from dash import Dash, dcc, html, Input, Output
from jupyter_dash import JupyterDash
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
covid_cases_data=pd.read_csv("covid_confirmed_usafacts.csv")
covid_cases_df=pd.DataFrame(covid_cases_data)
covid_deaths_data=pd.read_csv("covid_deaths_usafacts.csv")
covid_deaths_df=pd.DataFrame(covid_deaths_data)

In [ ]:
covid_cases_df.head()

In [ ]:
covid_deaths_df.head()

In [ ]:
# Filter data for the specified date range
start_date="2020-06-01"
end_date="2021-01-03"
date_range=pd.date_range(start_date, end_date).strftime('%Y-%m-%d').tolist()
covid_cases_data=covid_cases_data[['countyFIPS', 'County Name', 'State']+date_range]

date_range=pd.date_range(start_date, end_date).strftime('%Y-%m-%d').tolist()
covid_deaths_data=covid_deaths_data[['countyFIPS', 'County Name', 'State']+date_range]

covid_cases_data.head()

In [11]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go

# Read the COVID-19 cases data
covid_cases_data = pd.read_csv("covid_confirmed_usafacts.csv")

# Filter data for the specified date range
start_date = "2020-06-01"
end_date = "2021-01-03"
date_range = pd.date_range(start_date, end_date).strftime('%Y-%m-%d').tolist()
covid_cases_data = covid_cases_data[['countyFIPS', 'County Name', 'State'] + date_range]

# Prepare data for analysis
cases_data = covid_cases_data[date_range].sum(axis=0).values
days_since_start = np.arange(len(date_range)).reshape(-1, 1)

# Define the app
app = dash.Dash(__name__)

# App layout
app.layout = html.Div([
    html.H1("COVID-19 Cases and Deaths Prediction"),
    dcc.Graph(id='covid-graph'),
    html.Button('Linear Mode', id='linear-button', n_clicks=0),
    html.Button('Log Mode', id='log-button', n_clicks=0)
])

# Callback to update graph
@app.callback(
    Output('covid-graph', 'figure'),
    [Input('linear-button', 'n_clicks'),
     Input('log-button', 'n_clicks')]
)
def update_graph(linear_clicks, log_clicks):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    mode = None
    
    if 'linear-button' in changed_id:
        mode = "linear"
    elif 'log-button' in changed_id:
        mode = "log"

    # Apply mode transformation
    if mode == "log":
        cases_data_transformed = np.log(cases_data + 1)  # Adding 1 to avoid log(0)
    else:
        cases_data_transformed = cases_data

    # Fit regression model
    reg = LinearRegression().fit(days_since_start, cases_data_transformed)

    # Predictions
    predictions = reg.predict(days_since_start)
    if mode == "log":
        predictions = np.exp(predictions) - 1  # Transforming back from log space

    # Plot data
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=np.arange(len(date_range)), y=cases_data_transformed, mode='markers', name='Actual Cases'))
    fig.add_trace(go.Scatter(x=np.arange(len(date_range)), y=predictions, mode='lines', name='Trend Line'))
    fig.update_layout(title=f'COVID-19 Cases Prediction ({mode.capitalize() if mode else "Linear"} Mode)',
                      xaxis_title='Number of Days',
                      yaxis_title='Number of Cases' if mode == "linear" else 'Log(Number of Cases)',
                      legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
                      height=600)
    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8051)